<a href="https://colab.research.google.com/github/girisiman/Deep_Learning_Tasks/blob/master/DL_Task_1_Final_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras.datasets.mnist as mnist
import keras
from keras import regularizers
from keras.layers import Input
from keras.models import Model
from keras.layers import Layer
from keras.layers.core import Dense, Dropout
from keras.utils import np_utils
from keras.models import Sequential
from keras import optimizers
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import  KerasClassifier
import os
from sklearn.metrics import accuracy_score


Using TensorFlow backend.


Building all the function required to build a model, predict and table the required result

In [0]:
# function to build basic mlp:
def create_model(layers,activation, optimizer,drop_out, num_classes=10):
  ''' Build a simple mlp model for multi-class classification;  
      Arguments:all the hyperparametes (layers,act,opt, drop_out) to buld in a mlp provided by dictionary of parameters used during hyperparameter
      optimization
           '''
  model = Sequential()
  for i in range(len(layers)):
    if i == 0:
        model.add(Dense(layers[i], activation=activation, input_shape=(784,)))
        model.add(Dropout(drop_out))
    else:
      model.add(Dense(layers[i], activation=activation))
      model.add(Dropout(drop_out))
  model.add(Dense(num_classes, activation=activation))
  model.summary()

  model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

  return model

In [0]:
def build_classifier(model):
  ''' #classifier builder using keras_classifier
  #Argument: Model; the output of create_model,
  #This function handles the number of epochs provided for training
  #output, kerasclassifiermodel and dicitionary of parameter'''
  model = KerasClassifier(build_fn=create_model, epochs=25, batch_size=128, verbose=0)
  parameter = {'layers':[[512,512],[512,512,512]],'activation': [ 'elu',  'relu',],
             'optimizer': [ 'Adam' ],
             'drop_out': [ 0.5]}
  return keras_model, parameter



In [0]:
def search_besthyperparameter(model,parameter):
  ''' Function to search for best parameter using GridSearch CV
  #Argument:Kerasclassifiedmodel and dictionary of parameter
  #output Classifier after perforfing gridsearchCV and results of all the parameters
  #best_estimator: Estimators(with the combination of hyperparameter) providing best score
  ###Thus function trains the model and provide the result.'''
  classifier = GridSearchCV(estimator=model, param_grid=parameter, scoring ='accuracy')
  search_result = classifier.fit(x_train, y_train)
  print("Best Accuracy: %f using %s" % (search_result.best_score_, search_result.best_params_))
  best_estimator = classifier.best_estimator_
  return search_result, best_estimator

In [0]:
def table_search_result(grid_result):
  ''' Function to plot the top5 best parameter combination based on the gridsearchcv score
  #Argument:results from the training
  #output: table with top5 best parameter'''
  import pandas as pd
  means = grid_result.cv_results_['mean_test_score']
  #stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  rank = grid_result.cv_results_['rank_test_score']
  d = {'rank_param':rank,'means_test_score':means,  'params':params}
  result_table = pd.DataFrame(data =d)
  #display(result_table.head())
  sc = result_table.sort_values(by='rank_param',ascending = True)
  display(sc.head(10))
  
  

In [0]:
def predict_classifier(best_estimator,x_test,y_test):
  ''' Predict the accuracy of the classifier, using best found estimator
  #best_estimator: best found estimator
  #x_test,y_test: test datasets to predict'''
  #output_accuracy 
  #best_estimator = classifier.best_estimator_
  y_predicted = best_estimator.predict(x_test)
  print(y_predicted)
  accuracy = (accuracy_score(y_test,y_predicted)*100)
  print('Accuracy: ',accuracy*100)
  return accuracy

In [0]:
def load_data(num_classes = 10):
  ''' funtion that load original mnist dataset, seperated into train and test set
  #returns x_train,x_test,y_train,y_test'''
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train = x_train.reshape(60000, 784)
  x_test = x_test.reshape(10000, 784)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255
  print(x_train.shape[0], 'train samples')
  print(x_test.shape[0], 'test samples')
  return x_train, x_test, y_train, y_test

In [0]:
def add_noise(x_train, x_test):
  ''' Adds randomly generated gaussian noise to the train and test set
  # argument, x_train, x_test of original data
  #output: x_train and test with the nosise'''
  noise_factor = 0.5
  x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
  x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 
  x_train_noisy = np.clip(x_train_noisy, 0., 1.)
  x_test_noisy = np.clip(x_test_noisy, 0., 1.)
  return x_train_noisy, x_test_noisy

performing the classification with hyperparameter searching in the original data.

In [16]:
# loading the dataset:
x_train, x_test, y_train, y_test = load_data()

60000 train samples
10000 test samples


In [0]:
model = KerasClassifier(build_fn=create_model, epochs=25, batch_size=128, verbose=0)
parameter = {'layers':[[512,512],[512,512,512]],'activation': [ 'elu',  'relu',],
             'optimizer': [ 'Adam' ],
             'drop_out': [ 0.5]}

In [19]:
search_result,best_estimator = search_besthyperparameter(model,parameter)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)          

In [25]:
table_search_result(search_result)

,rank_param,means_test_score,params
1,1,0.912167,"{'activation': 'elu', 'drop_out': 0.5, 'layers..."
0,2,0.827150,"{'activation': 'elu', 'drop_out': 0.5, 'layers..."
2,3,0.098717,"{'activation': 'relu', 'drop_out': 0.5, 'layer..."
3,3,0.098717,"{'activation': 'relu', 'drop_out': 0.5, 'layer..."


In [38]:
Result={'Accuracy_original':[],'Accuracy_noise':[],'Accuracy_recon':[]}
Accuracy_original = predict_classifier(best_estimator,x_test,y_test)
Result.update(Accuracy_original = Accuracy_original)
Result

[7 2 1 ... 4 5 6]
Accuracy:  9274.0


{'Accuracy_noise': [], 'Accuracy_original': 92.74, 'Accuracy_recon': []}

In [0]:
# loading the dataset with noise:
x_train_noise, x_test_noise = add_noise(x_train,x_test)

In [34]:
# training and hyperparameter searaching with model and parameter
search_result_noise,best_estimator_noise = search_besthyperparameter(model,parameter)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_47 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_34 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_35 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dens

In [35]:
table_search_result(search_result)

,rank_param,means_test_score,params
1,1,0.912167,"{'activation': 'elu', 'drop_out': 0.5, 'layers..."
0,2,0.827150,"{'activation': 'elu', 'drop_out': 0.5, 'layers..."
2,3,0.098717,"{'activation': 'relu', 'drop_out': 0.5, 'layer..."
3,3,0.098717,"{'activation': 'relu', 'drop_out': 0.5, 'layer..."


In [39]:
Accuracy_noise = predict_classifier(best_estimator,x_test,y_test)
Result.update(Accuracy_noise = Accuracy_noise)
Result

[7 2 1 ... 4 5 6]
Accuracy:  9274.0


{'Accuracy_noise': 92.74, 'Accuracy_original': 92.74, 'Accuracy_recon': []}